In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class CNN1D(nn.Module):
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 15, 120)  # This needs to be adjusted based on your input size
        self.fc2 = nn.Linear(120, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Adding a channel dimension
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Assuming your input length is 63
seq_length = 63
num_classes = 13  # Change this to your actual number of classes
model = CNN1D(num_classes=num_classes)

# Dummy input corresponding to one sequence
# The input size should be (batch_size, seq_length), here batch_size is 1
dummy_input = Variable(torch.randn(1, seq_length))

# Set the model to evaluation mode
model.eval()

# Export the model
torch.onnx.export(model,                 # model being run
                  dummy_input,           # model input (or a tuple for multiple inputs)
                  "cnn1d_model.onnx",    # where to save the model
                  export_params=True,    # store the trained parameter weights inside the model file
                  opset_version=10,      # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],    # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [3]:
import onnxruntime as ort

# Load the ONNX model using ONNX Runtime
ort_session = ort.InferenceSession("cnn1d_model.onnx")

# Prepare the input data as a dictionary
inputs = {ort_session.get_inputs()[0].name: dummy_input.numpy()}

# Run inference
outputs = ort_session.run(None, inputs)

# Print the output to verify
print(outputs)

[array([[-0.13178971, -0.03653295,  0.008347  ,  0.0379477 ,  0.0635919 ,
        -0.09029373,  0.02039379,  0.07009204, -0.01618053,  0.03866152,
         0.09614401,  0.02165812, -0.04923753]], dtype=float32)]
